In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import re
from sklearn.pipeline import Pipeline
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
import pickle
import tweepy

In [8]:
user = 'ns_mia'

In [2]:
# create connection with Twitter API

CONSUMER_KEY = 'J3uhwWHdmSt3uD69ry8r2kc3B'
CONSUMER_SECRET = 'LeuM7dVfFbXH0bc5fFkZIxlNiUXyjIx4Kcjh58HeWs2TrLijDx'
ACCESS_TOKEN = '327497511-5dNjiGaTQHASljoxNr1qlMOlHrrSB21HBFBDjx2E'
ACCESS_TOKEN_SECRET = 'tHLqwlhHYJWptW3femyq0rbMW6ZItu5yLQ3DFGJqJ8Xlg'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [11]:
stuff = api.user_timeline(, count = 100, include_rts = True, tweet_mode="extended")
tweets = []

for tweet in stuff:
    tweets.append(tweet._json['full_text'])
    
tweets = pd.Series(tweets)

In [12]:
tweets

0                                            @DibeBot a
1                                              Testing!
2     I just downloaded Blanka, a new free font by @...
3     Con My chance ho provato a vincere i biglietti...
4     We opened the seven club tonight, good job tea...
5     RT @the_monz94: Big night #serataalette #KFC #...
6     Shooting training tonight, ready for #Cinisell...
7     Chef @teony7 is doing a great job!! #ChickenWi...
8     Thanks @luigi_banfi, We'll do our best! See yo...
9     Having fun the night before the most important...
10    Intense workout tonight, in preparation for th...
11         @JackB_OS @PowZerR tutto naturale e genuino!
12    RT @parallelecinico: Lo sport al quale gioca G...
13    We came back to win last night, but we have to...
14    Ready for the game! #CesateBasketball #StriveF...
15    RT @the_monz94: Only best player have training...
16    Long study session in l26 building tonight, we...
17    Bad beat tonight, it's time to rise up now

In [5]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc:(stemmer.stem(w) for w in analyzer(doc))

In [6]:
nb = pickle.load( open( "nb.model", "rb" ) )

In [7]:
def remove_rt(x):
    
    if 'RT' in x:
        x = x.replace('RT', '')
        try:
            return x[x.rindex(':')+2:]
        except:
            return x
    else:
        return x

In [8]:
stop_words = stopwords.words('english')

def remove_stop(x):
    return [word for word in x.split() if word not in stop_words]

In [9]:
tweets = tweets.apply(lambda x: remove_rt(x))
tweets = tweets.apply(lambda x: re.sub(r'^\/t.co\/[^\s]+', '', x))
tweets = tweets.apply(lambda x: re.sub(r'[^\w\s]','',x))
tweets = tweets.apply(lambda x: x.lower())
tweets = tweets.apply(lambda x: remove_stop(x))
tweets = tweets.astype(str)
tweets = tweets[tweets!='[]']

- predict proba
- get predictions avg
- return [p1,p2,p3,p4,p5]

In [21]:
pred = nb.predict_proba(tweets)

In [27]:
np.mean(pred, axis=0).tolist()

[0.2911060882691445,
 0.2004621682673945,
 0.2521561100848258,
 0.078798418405524,
 0.17747721497311147]